In [ ]:
import os
MODIS_dir="/content/drive/MyDrive/Data_county_pull_modis_entire_country"
print(len([name for name in os.listdir(MODIS_dir) if os.path.isfile(os.path.join(MODIS_dir, name))]))

1834


In [ ]:
!pip install 2to3
!pip install absl-py
!pip install appnope
!pip install asn1crypto
!pip install astor
!pip install backcall
!pip install beautifulsoup4
!pip install bleach
!pip install cachetools
!pip install certifi
!pip install cffi
!pip install cloudpickle
!pip install cryptography
!pip install cycler
!pip install cytoolz
!pip install dask
!pip install decorator
!pip install earthengine-api
!pip install gast
!pip install GDAL
!pip install google-api-python-client
!pip install google-auth
!pip install google-auth-httplib2
!pip install grpcio
!pip install h5py
!pip install html5lib
!pip install httplib2
!pip install idna
!pip install imageio
!pip install ipython
!pip install ipython-genutils
!pip install jedi
!pip install joblib
!pip install Keras-Applications
!pip install Keras-Preprocessing
!pip install kiwisolver
!pip install Markdown
!pip install matplotlib
!pip install mkl-fft
!pip install mkl-random
!pip install networkx
!pip install numpy
!pip install oauth2client
!pip install olefile
!pip install pandas
!pip install parso
!pip install pexpect
!pip install pickleshare
!pip install Pillow
!pip install prompt-toolkit
!pip install protobuf
!pip install ptyprocess
!pip install pyasn1
!pip install pyasn1-modules
!pip install pycparser
!pip install Pygments
!pip install pyOpenSSL
!pip install pyparsing
!pip install python-dateutil
!pip install pytz
!pip install PyWavelets
!pip install rsa
!pip install scikit-image
!pip install scikit-learn
!pip install scipy
!pip install six
!pip install soupsieve
!pip install tensorboard
!pip install tensorflow
!pip install tensorflow-tensorboard
!pip install termcolor
!pip install toolz
!pip install tornado
!pip install traitlets
!pip install uritemplate
!pip install wcwidth
!pip install Werkzeug
!pip install xarray

     |████████████████████████████████| 112kB 5.0MB/s 
     |████████████████████████████████| 3.2MB 6.6MB/s 
     |████████████████████████████████| 481kB 5.1MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.0-cp37-cp37m-linux_x86_64.whl size=1224569 sha256=5e953a5e9f0be470ccc7695057f0a1341e629dafe17c2cdaf85c2eca56d4b63e
  Stored in directory: /root/.cache/pip/wheels/a1/32/3c/9c9926b510647cacdde744b2c7acdf1ccd5896fbb7f8d5df0c
Successfully built cytoolz
     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 245kB 5.2MB/s 
     |████████████████████████████████| 3.5MB 32.8MB/s 
ERROR: Could not find a version that satisfies the requirement intel-opencl-rt==2021.3.0 (from dpcpp_cpp_rt->mkl-fft) (from versions: none)
ERROR: No matching distribution found for intel-opencl-rt==2021.3.0 (from dpcpp_cpp_rt->mkl-fft)
     |████████████████████████████████| 389kB 5.1MB/s 
  Using cached https://files.pythonhosted.org/packages/b5/19/9bae82954fb3697b9

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib.request, urllib.parse, urllib.error

In [ ]:
import ee
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=dSPHirWh5UJ6L-albOl2psgF-NKxXQNZ3kpMxUsTH6s&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgy9aujnD4frTUEabL6WV9kaw2-shbMlzWS-Nsok4uyTm8vVQV8FSg

Successfully saved authorization token.


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
ee.Initialize()
print(ee)

<module 'ee' from '/usr/local/lib/python3.7/dist-packages/ee/__init__.py'>


In [ ]:
def export_oneimage(img,folder,name,region,scale,crs):
  task = ee.batch.Export.image(img, name, {
      'driveFolder':folder,
      'driveFileNamePrefix':name,
      'region': region,
      'scale':scale,
      'crs':crs
  })
  task.start()
  while task.status()['state'] == 'RUNNING':
    print('Running...')
    # Perhaps task.cancel() at some point.
    time.sleep(10)
  print('Done.', task.status())

In [ ]:
locations = pd.read_csv('/content/drive/MyDrive/Crop_Yield/locations_final.csv')

In [ ]:
def appendBand(current, previous):
    # Rename the band
    #print("Before accum")
    #previous=ee.Image(previous)
    #current = current.select([0,1,2,3,4,5,6])
    # # Append it to the result (Note: only return current item on first element/iteration)
    #accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current),overwrite=True))
    print("Before accum")
    #accum = ee.Image(previous).addBands(current.select([0,1,2,3,4,5,6]))
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(ee.Image(previous),None), current.select([0,1,2,3,4,5,6]), ee.Image(previous).addBands(ee.Image(current.select([0,1,2,3,4,5,6]))))
    #ee.Image(previous).addBands(image)
    #print("accum = ",accum)
    # Return the accumulation
    return accum

In [ ]:
county_region = ee.FeatureCollection('ft:18Ayj5e7JxxtTPm1BdMnnzWbZMrxMB49eqGDTsaSp')

In [ ]:
imgcoll = ee.ImageCollection('MODIS/MOD09A1').filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23)).filterDate('2001-12-31','2015-12-31')
# print("Image collection - ",imgcoll.size())
count = imgcoll.size()
print('Count: ', str(count.getInfo())+'\n')
# MDT
first = imgcoll.first()
#img=imgcoll.iterate(appendBand,first)
#img=imgcoll.iterate(appendBand)
#img = ee.Image(imgcoll.iterate(appendBand,first))
img = ee.Image(imgcoll.iterate(appendBand))
img=ee.Image(img)

Count:  644

Before accum
Before accum


In [ ]:
img_0=ee.Image(ee.Number(0))
img_5000=ee.Image(ee.Number(5000))

img=img.min(img_5000)
img=img.max(img_0)

In [ ]:
for loc1, loc2, lat, lon in locations[:].values:
    fname = '{}_{}'.format(int(loc1), int(loc2))

    offset = 0.11
    scale  = 500
    crs='EPSG:4326'

    # filter for a county
    #region = county_region.filterMetadata('STATE num', 'equals', loc1)
    #region = ee.FeatureCollection(region).filterMetadata('COUNTY num', 'equals', loc2)
    #region = region.first()
    #region = region.geometry().coordinates().getInfo()[0]

    region = str([
         [lat - offset, lon + offset],
         [lat + offset, lon + offset],
         [lat + offset, lon - offset],
         [lat - offset, lon - offset]])
    #while True:
    #try:
    export_oneimage(img, 'Final1_Data_county_pull_modis_entire_country', fname, region, scale, crs)
    #except:
    #  print('retry')
    #        time.sleep(10)
    #        continue
    #break

Done. {'state': 'READY', 'description': '1_65', 'creation_timestamp_ms': 1625062029665, 'update_timestamp_ms': 1625062029665, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'LXJD3A6XWWTC5HRZMSRUTARR', 'name': 'projects/earthengine-legacy/operations/LXJD3A6XWWTC5HRZMSRUTARR'}
Done. {'state': 'RUNNING', 'description': '1_85', 'creation_timestamp_ms': 1625062030489, 'update_timestamp_ms': 1625062231272, 'start_timestamp_ms': 1625062047628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GSQJIAZ2FMLRKAOUDQCLP42I', 'name': 'projects/earthengine-legacy/operations/GSQJIAZ2FMLRKAOUDQCLP42I'}
Done. {'state': 'RUNNING', 'description': '1_105', 'creation_timestamp_ms': 1625062268585, 'update_timestamp_ms': 1625062411447, 'start_timestamp_ms': 1625062288389, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'OWE5SX4UE3PDAOZTHQBI2DNR', 'name': 'projects/earthengine-legacy/operations/OWE5SX4UE3PDAOZTHQBI2DNR'}
Done. {'state': 'READY', 'description': '1_119', 'creation_timestamp_ms': 16